# Image subtraction code for CT2B
## Input images
- `A1`: Time point 1; Green channel (AF488)
- `B1`: Time point 1; Yellow channel (CY3)
- `C1`: Time point 1; Red channel (CY5)
- `D1`: Time point 1; Merged channel (AF488 + CY3 + CY5)
-  
- `A2`: Time point 2; Green channel (AF488)
- `B2`: Time point 2; Yellow channel (CY3)
- `C2`: Time point 2; Red channel (CY5)
- `D2`: Time point 2; Merged channel (AF488 + CY3 + CY5)
-    
- `A3`: Time point 3; Green channel (AF488)
- `B3`: Time point 3; Yellow channel (CY3)
- `C3`: Time point 3; Red channel (CY5)
- `D3`: Time point 3; Merged channel (AF488 + CY3 + CY5)

Time point 1 is the reference image for alignment
## Image alignment

In [3]:
import cv2
import numpy as np

def load_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Unable to load image at {image_path}. Check the file path and integrity.")
    else:
        # Optionally resize the image to reduce memory usage
        print(f"Successfully loaded image from {image_path} with shape {image.shape}")
    return image

def align_images(base_image, target_image):
    if base_image is None or target_image is None:
        print("Error: One of the images is not loaded.")
        return None, None

    try:
        # Convert images to grayscale
        base_gray = cv2.cvtColor(base_image, cv2.COLOR_BGR2GRAY)
        target_gray = cv2.cvtColor(target_image, cv2.COLOR_BGR2GRAY)

        # Initialize SIFT detector
        sift = cv2.SIFT_create()

        # Find the keypoints and descriptors with SIFT
        kp1, des1 = sift.detectAndCompute(base_gray, None)
        kp2, des2 = sift.detectAndCompute(target_gray, None)

        # Check if descriptors are found
        if des1 is None or des2 is None:
            print("Error: Descriptors not found in one of the images.")
            return None, None

        # FLANN parameters
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)

        # Initialize FLANN matcher
        flann = cv2.FlannBasedMatcher(index_params, search_params)

        # Match descriptors using KNN
        matches = flann.knnMatch(des1, des2, k=2)

        # Store good matches as per Lowe's ratio test.
        good_matches = []
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good_matches.append(m)

        if len(good_matches) > 4:
            src_pts = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

            # Calculate Homography
            H, _ = cv2.findHomography(dst_pts, src_pts, cv2.RANSAC, 5.0)
            # Warp target image to align with the base image
            aligned_image = cv2.warpPerspective(target_image, H, (base_image.shape[1], base_image.shape[0]))
            return aligned_image, H
        else:
            print("Error: Not enough matches found.")
            return None, None
    except cv2.error as e:
        print(f"OpenCV error: {e}")
        return None, None

# Load images
D1 = load_image(r'D://testcode//D1.tif')
D2 = load_image(r'D://testcode//D2.tif')
D3 = load_image(r'D://testcode//D3.tif')

# Align D2 and D3 to D1 if loaded successfully
if D1 is not None and D2 is not None:
    aligned_D2, H2 = align_images(D1, D2)
    if aligned_D2 is None:
        print("Failed to align D2. Exiting.")
        exit()

if D1 is not None and D3 is not None:
    aligned_D3, H3 = align_images(D1, D3)
    if aligned_D3 is None:
        print("Failed to align D3. Exiting.")
        exit()

# Load channels for D2 (A2, B2, C2)
A2 = load_image(r'D://testcode//A2.tif')
B2 = load_image(r'D://testcode//B2.tif')
C2 = load_image(r'D://testcode//C2.tif')

if A2 is not None and B2 is not None and C2 is not None:
    # Apply the same transformation to channels
    aligned_A2 = cv2.warpPerspective(A2, H2, (D1.shape[1], D1.shape[0]))
    aligned_B2 = cv2.warpPerspective(B2, H2, (D1.shape[1], D1.shape[0]))
    aligned_C2 = cv2.warpPerspective(C2, H2, (D1.shape[1], D1.shape[0]))
    print("Aligned channels for D2 successfully.")
else:
    print("Failed to load one or more channels for D2. Exiting.")
    exit()

# Load channels for D3 (A3, B3, C3)
A3 = load_image(r'D://testcode//A3.tif')
B3 = load_image(r'D://testcode//B3.tif')
C3 = load_image(r'D://testcode//C3.tif')

if A3 is not None and B3 is not None and C3 is not None:
    # Apply the same transformation to channels
    aligned_A3 = cv2.warpPerspective(A3, H3, (D1.shape[1], D1.shape[0]))
    aligned_B3 = cv2.warpPerspective(B3, H3, (D1.shape[1], D1.shape[0]))
    aligned_C3 = cv2.warpPerspective(C3, H3, (D1.shape[1], D1.shape[0]))
    print("Aligned channels for D3 successfully.")
else:
    print("Failed to load one or more channels for D3. Exiting.")
    exit()



Successfully loaded image from D://testcode//D1.tif with shape (1024, 1024, 3)
Successfully loaded image from D://testcode//D2.tif with shape (1024, 1024, 3)
Successfully loaded image from D://testcode//D3.tif with shape (1024, 1024, 3)
Successfully loaded image from D://testcode//A2.tif with shape (1024, 1024, 3)
Successfully loaded image from D://testcode//B2.tif with shape (1024, 1024, 3)
Successfully loaded image from D://testcode//C2.tif with shape (1024, 1024, 3)
Aligned channels for D2 successfully.
Successfully loaded image from D://testcode//A3.tif with shape (1024, 1024, 3)
Successfully loaded image from D://testcode//B3.tif with shape (1024, 1024, 3)
Successfully loaded image from D://testcode//C3.tif with shape (1024, 1024, 3)
Aligned channels for D3 successfully.


## Save aligned image

In [4]:
# Save aligned images
if aligned_D2 is not None:
    cv2.imwrite('D://testcode//aligned_D2.tif', aligned_D2)
    print("Aligned D2 saved as 'aligned_D2.jpg'.")

if aligned_D3 is not None:
    cv2.imwrite('D://testcode//aligned_D3.tif', aligned_D3)
    print("Aligned D3 saved as 'aligned_D3.jpg'.")

if aligned_A2 is not None:
    cv2.imwrite('D://testcode//aligned_A2.tif', aligned_A2)
    print("Aligned A2 saved as 'aligned_A2.jpg'.")

if aligned_B2 is not None:
    cv2.imwrite('D://testcode//aligned_B2.tif', aligned_B2)
    print("Aligned B2 saved as 'aligned_B2.jpg'.")

if aligned_C2 is not None:
    cv2.imwrite('D://testcode//aligned_C2.tif', aligned_C2)
    print("Aligned C2 saved as 'aligned_C2.jpg'.")

if aligned_A3 is not None:
    cv2.imwrite('D://testcode//aligned_A3.tif', aligned_A3)
    print("Aligned A3 saved as 'aligned_A3.jpg'.")

if aligned_B3 is not None:
    cv2.imwrite('D://testcode//aligned_B3.tif', aligned_B3)
    print("Aligned B3 saved as 'aligned_B3.jpg'.")

if aligned_C3 is not None:
    cv2.imwrite('D://testcode//aligned_C3.tif', aligned_C3)
    print("Aligned C3 saved as 'aligned_C3.jpg'.")


Aligned D2 saved as 'aligned_D2.jpg'.
Aligned D3 saved as 'aligned_D3.jpg'.
Aligned A2 saved as 'aligned_A2.jpg'.
Aligned B2 saved as 'aligned_B2.jpg'.
Aligned C2 saved as 'aligned_C2.jpg'.
Aligned A3 saved as 'aligned_A3.jpg'.
Aligned B3 saved as 'aligned_B3.jpg'.
Aligned C3 saved as 'aligned_C3.jpg'.


## Resolve proteins according to the pre-set color barcode 
### The following is an example for SV2

In [8]:
# Import images
ImageC1 = cv2.imread('D://testcode//C1.tif', cv2.IMREAD_GRAYSCALE)
ImageC2 = cv2.imread('D://testcode//aligned_C2.tif', cv2.IMREAD_GRAYSCALE)

# Get the size of each image
height, width = ImageC1.shape

# Create an empty matrix to store the new gray value
new_gray_values = np.zeros((height, width), dtype=np.uint16)

# Traverse each pixel coordinate
for i in range(height):
    for j in range(width):
        # Calculate the new gray value according to the pre-set color barcode (the following is just an example for SV2)
        if ImageC1[i, j] > ImageC2[i, j]:
            new_gray_values[i, j] = ImageC1[i, j] - ImageC2[i, j]
        else:
            new_gray_values[i, j] = 0
# Convert the new grayscale value matrix to an image
cv2.imwrite('D://testcode//SV2.tif', new_gray_values)

True

## Image display
The decoded images were imported to zen 3.11 software for viewing. 

The grayscale image was assigned with pseudo color:  Open zen 3.11 → Dimensions → Channels.

To merge images: Open zen 3.11 → Processing → Method → Add channes.